In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
from pandas.api.types import is_numeric_dtype

In [44]:
def cardinalidad(df_in, umbral_categoria = 10, umbral_countinua = 30):
    cardinalidad = [df_in[col].nunique() for col in df_in.columns]
    cardinalidad_por = [df_in[col].nunique()/len(df_in[col]) for col in df_in.columns]
    dict_df = {"nombre_variable": df_in.columns, "valores_unicos": cardinalidad, "cardinalidad": cardinalidad_por}
    nuevo_df = pd.DataFrame(dict_df)
    nuevo_df["tipo_sugerido"] = "Categórica"
    nuevo_df.loc[nuevo_df["valores_unicos"] == 2, "tipo_sugerido"] = "Binaria"
    nuevo_df.loc[nuevo_df["valores_unicos"] >= umbral_categoria, "tipo_sugerido"] = "Numerica Discreta"
    nuevo_df.loc[nuevo_df["cardinalidad"] >= umbral_countinua, "tipo_sugerido"] = "Numerica Continua"
    return nuevo_df

In [45]:
def get_features_cat_regression(df, target_col, p_value = 0.05, umbral_cat = 10):
    '''
    Esta función filtra las variables categóricas de un dataset introducido para entrenar un modelo de regresión lineal. 
    Verifica el tipo de variable llamando a la función "cardinalidad".
    Si la variable es binaria aplica el test Mann-Whitney U, si es categórica pero no binaria aplica el test ANOVA para comprobar su relación con la variable target.
    Si el valor p de los tests está por debajo del umbral especificado en "p_value" puede descartarse la hipótesis nula (la variable target y la categórica no estan relacionadas) con confianza estadística y se añade la variable a la lista de features categóricas para el modelo.


    
    Argumentos:

    df(pd.DataFrame): DataFrame cuyas variables categóricas se desea filtrar.

    target_col(string): nombre de la columna target que se pretende predecir con el modelo.

    p_value(float): umbral de valor p por debajo del cual debe estar el valor p del test aplicado para determinar la relación entre una variable y el target para añadir dicha variable a la lista de features categóricas.
    
    umbral_cat(int): controla el número máximo de valores que puede tener una columna para ser considerada categórica.

    

    Retorna:

    list: Lista de features categóricas para entrenar un modelo de regresión lineal con el dataset dado.
    '''
    if not is_numeric_dtype(df[target_col]):
        target_col = input("Tu variable objetivo no es de tipo numérico, introduce una nueva variable target o la palabra 'parar' para dejar de ejecutar la función.")
        if target_col == "parar":
            return "La función no se ha ejecutado porque has decidido pararla"
    if len(df.loc[df[target_col].isna()]) > 0:
        raise Exception(f"La variable '{target_col}' tiene valores nulos, introduce una variable target sin nulos")    
    from scipy.stats import mannwhitneyu
    from scipy.stats import f_oneway
    lista_cat = []
    for col in df:
        if len(df.loc[df[col].isna()]) > 0:
            raise Exception(f"La variable '{col}' tiene valores nulos, introduce un DataFrame sin nulos")
        tipo_col = cardinalidad(df[[col]], umbral_categoria = umbral_cat).tipo_sugerido[0]
        if tipo_col == "Binaria":
            value_1 = df[col].unique()[0]
            value_2 = df[col].unique()[1]
            group_a = df.loc[df[col] == value_1, target_col]
            group_b = df.loc[df[col] == value_2, target_col]
            _, p_val = mannwhitneyu(group_a, group_b)
            print(f"Para '{target_col}' y '{col}' el p-value es: {p_val} (Test realizado: Mann-Whitney U)")
            print(p_val)
            if p_val < p_value:
                lista_cat.append(col)
        elif tipo_col == "Categórica":
            groups = df[col].unique()
            target_values_x_group = [df.loc[df[col] == group, target_col] for group in groups]
            _, p_val = f_oneway(*target_values_x_group)
            print(f"Para '{target_col}' y '{col}' el p-value es: {p_val} (Test aplicado: ANOVA)")
            if p_val < p_value:
                lista_cat.append(col)
    return lista_cat

In [46]:
def plot_features_cat_regression(df, target_col="", columns=[], pvalue=0.05, with_individual_plot=False, umbral_cat=10):
    
    """
    Pinta histogramas agrupados de columnas categóricas que muestran relación estadísticamente significativa
    con una variable target numérica. Usa Mann-Whitney U para 2 grupos y ANOVA para más.

    Argumentos:
    df (pd.DataFrame): El DataFrame con los datos.
    target_col (str): Nombre de la columna objetivo (debe ser numérica continua o discreta con alta cardinalidad).
    columns (list of str): Lista de nombres de columnas categóricas a analizar. Si está vacía, se seleccionan automáticamente.
    pvalue (float): Nivel de significación estadística para los test (por defecto 0.05).
    with_individual_plot (bool): Si es True, se muestran gráficos individuales por variable.

    Devuelve:
    list of str | None: Lista de variables categóricas con relación estadísticamente significativa, o None si hay error.
    """

    # Comprobamos: Si es un DataFrame
    
    if not isinstance(df, pd.DataFrame):
        print("Error: 'df' debe ser un DataFrame de pandas.")
        return None

    # Comprobamos: Si target_col es una columna en el DataFrame y si no es un string
    if not isinstance(target_col, str) or target_col not in df.columns:
        print("Error: 'target_col' debe ser una columna válida del DataFrame.")
        return None
    if not np.issubdtype(df[target_col].dtype, np.number):
        print("Error: 'target_col' debe ser numérico.")
        return None

    if df[target_col].isna().sum() > 0:
        print(f"Error: La variable '{target_col}' tiene valores nulos.")
        return None

    if df[target_col].nunique() < 10:
        print("Error: 'target_col' debe tener alta cardinalidad (mínimo 10 valores únicos).")
        return None

    if not isinstance(columns, list) or not all(isinstance(col, str) for col in columns):
        print("Error: 'columns' debe ser una lista de strings.")
        return None

    if not isinstance(pvalue, (float, int)) or not (0 < pvalue < 1):
        print("Error: 'pvalue' debe estar entre 0 y 1.")
        return None

    if not isinstance(with_individual_plot, bool):
        print("Error: 'with_individual_plot' debe ser booleano.")
        return None


      # Si no se especifican columnas,  las obtenemos automáticamente
    if not columns:
        columns = get_features_cat_regression(df, target_col, p_value=pvalue, umbral_cat=umbral_cat)
    else:
        # Validar columnas pasadas y filtrar por significancia
        valid_columns = [col for col in columns if col in df.columns]
        columns = [col for col in valid_columns
                   if col in get_features_cat_regression(df[[col, target_col]], target_col, p_value=pvalue, umbral_cat=umbral_cat)]

    if not columns:
        print("No se encontraron variables categóricas con relación significativa con el target.")
        return []

    # Graficar si se solicita
    for col in columns:
        if with_individual_plot:
            plt.figure(figsize=(8, 4))
            sns.histplot(data=df, x=target_col, hue=col, multiple="stack", kde=True)
            plt.title(f"{target_col} por {col}")
            plt.tight_layout()
            plt.show()

    return columns

In [47]:
df = pd.read_csv("./data/titanic.csv")
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [48]:
get_features_cat_regresssion(df, "age")

[]

In [49]:
plot_features_cat_regression(df, "fare", with_individual_plot = True)

Para 'fare' y 'survived' el p-value es: 4.553477179250238e-22 (Test realizado: Mann-Whitney U)
4.553477179250238e-22
Para 'fare' y 'pclass' el p-value es: 1.0313763209142051e-84 (Test aplicado: ANOVA)
Para 'fare' y 'sex' el p-value es: 9.612326962909258e-15 (Test realizado: Mann-Whitney U)
9.612326962909258e-15


Exception: La variable 'age' tiene valores nulos, introduce un DataFrame sin nulos

In [50]:
plot_features_cat_regression(df, "age", with_individual_plot = True)

Error: La variable 'age' tiene valores nulos.
